# **Sentiment Analysis**

##  Subreddit Post Sentiment Classification



In [1]:
#importing librarires
import pandas as pd    #linear algebra
import numpy as np     #data processing

train_data = pd.read_json("https://raw.githubusercontent.com/rpsoft/tad_course/main/reddit_sentiment_train.json")

validation_data = pd.read_json("https://raw.githubusercontent.com/rpsoft/tad_course/main/reddit_sentiment_validation.json")

test_data = pd.read_json("https://raw.githubusercontent.com/rpsoft/tad_course/main/reddit_sentiment_test.json")

In [2]:
train_data.head(5)

,subreddit,title,id,url,author,body,majority_type,is_first_post,post_depth,in_reply_to,sentiment.polarity,sentiment.subjectivity
0,relationships,My Friend/Crush [22/F] acting weird after I [2...,t1_cy7f317,https://www.reddit.com/r/relationships/comment...,Melodrama_,"It's a sad realization, isn't it?",,False,2,t1_cy7erc5,negative,1.000000
1,relationships,My Friend/Crush [22/F] acting weird after I [2...,t1_cy7hlyf,https://www.reddit.com/r/relationships/comment...,Melodrama_,I told her a couple of minutes ago that I didn...,elaboration,False,2,t1_cy7erc5,neutral,0.483631
2,relationships,My Friend/Crush [22/F] acting weird after I [2...,t1_cy7etrr,https://www.reddit.com/r/relationships/comment...,TreatYoSelves,Leeches don't make good friends.,answer,False,1,t3_3xshx9,positive,0.600000
3,relationships,My Friend/Crush [22/F] acting weird after I [2...,t1_cy7hhpq,https://www.reddit.com/r/relationships/comment...,Melodrama_,I just ended it. Apparently she wasn't a good ...,elaboration,False,2,t1_cy7etrr,positive,0.475000
4,relationships,My Friend/Crush [22/F] acting weird after I [2...,t1_cy7q0qg,https://www.reddit.com/r/relationships/comment...,TreatYoSelves,Good for you! Make sure you stick with it.,appreciation,False,3,t1_cy7hhpq,positive,0.744444


In [3]:
#to extract information, pre-processing
import spacy

# Load the medium english model. 
# We will use this model to get embedding features for tokens later.
#!python -m spacy download en_core_web_md
nlp = spacy.load('en_core_web_sm', disable=['ner'])
nlp.remove_pipe('tagger')
nlp.remove_pipe('parser')

#importing nltk
# Download a stopword list
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
#funtion to Tokenize
def spacy_tokenize(string):
  tokens = list()
  doc = nlp(string)
  for token in doc:
    tokens.append(token)
  return tokens

#funtion to Normalize
def normalize(tokens):
  normalized_tokens = list()
  for token in tokens:
    normalized = token.text.lower().strip()
    if ((token.is_alpha or token.is_digit)):
      normalized_tokens.append(normalized)
  return normalized_tokens
  return normalized_tokens

#Tokenize and normalize
def tokenize_normalize(string):
  return normalize(spacy_tokenize(string))

#Q1
Use the text from the reddit posts (Known as “body”) to train classification models using the Scikit Learn package.
The labels to predict are the sentiment.polarity for each post. Conduct experiments using the following combinations
of classifier models and feature representations:


(a) Dummy Classifier with strategy="most_frequent"

(b) Dummy Classifier with strategy="stratified"

(c) LogisticRegression with One-hot vectorization

(d) LogisticRegression with TF-IDF vectorization (default settings)

(e) SVC Classifier with One-hot vectorization (SVM with RBF kernel, default settings))


(f) An ‘interesting’ classifier model and vectorization of your choice with appropriate pre-processing New Section



In [5]:
#Converting into Bag of Words(vectorization)
from sklearn.feature_extraction.text import CountVectorizer

# Pass in the tokenizer as the tokenizer to the vectorizer.
# Create a one-hot encoding vectorizer.
vectorizer = CountVectorizer(tokenizer=tokenize_normalize, binary=True)
train_features = vectorizer.fit_transform(train_data['body'])

# This creates input features for our classification on all subsets of our collection.
validation_features = vectorizer.transform(validation_data['body'])
test_features = vectorizer.transform(test_data['body'])

In [6]:

train_labels = train_data['sentiment.polarity']
validation_labels = validation_data['sentiment.polarity']
test_labels = test_data['sentiment.polarity']



In [7]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import fbeta_score

def evaluation_summary(description, predictions, true_labels):
  print("Evaluation for: " + description)
  precision = precision_score(predictions, true_labels,average=None)
  recall = recall_score(predictions, true_labels,average=None)
  accuracy = accuracy_score(predictions, true_labels)
  f1 = fbeta_score(predictions, true_labels, 1,average=None) #1 means f_1 measure
  print("Classifier has Acc=", (description,accuracy,precision,recall,f1))
  print(classification_report(predictions, true_labels, digits=3, zero_division = 0))
  print('\nConfusion matrix:\n',confusion_matrix(true_labels, predictions)) # Note the order here is true, predicted


In [8]:
##Solution (a)
from sklearn.dummy import DummyClassifier


dummy_mf = DummyClassifier(strategy='most_frequent')
dummy_mf.fit(train_features, train_labels)
print(dummy_mf.score(validation_features, validation_labels))
evaluation_summary("Dummy Majority", dummy_mf.predict(validation_features), validation_labels)


0.6307494371180444
Evaluation for: Dummy Majority
Classifier has Acc= ('Dummy Majority', 0.6307494371180444, array([0., 1., 0., 0., 0.]), array([0.        , 0.63074944, 0.        , 0.        , 0.        ]), array([0.        , 0.77357002, 0.        , 0.        , 0.        ]))
               precision    recall  f1-score   support

     negative      0.000     0.000     0.000         0
      neutral      1.000     0.631     0.774      3109
     positive      0.000     0.000     0.000         0
very negative      0.000     0.000     0.000         0
very positive      0.000     0.000     0.000         0

     accuracy                          0.631      3109
    macro avg      0.200     0.126     0.155      3109
 weighted avg      1.000     0.631     0.774      3109


Confusion matrix:
 [[   0  215    0    0    0]
 [   0 1961    0    0    0]
 [   0  845    0    0    0]
 [   0   15    0    0    0]
 [   0   73    0    0    0]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
#solution (b)
dummy_prior = DummyClassifier(strategy='stratified')
dummy_prior.fit(train_features, train_labels)
print(dummy_prior.score(validation_features, validation_labels))
evaluation_summary("Dummy Prior", dummy_prior.predict(validation_features), validation_labels)

0.4509488581537472
Evaluation for: Dummy Prior
Classifier has Acc= ('Dummy Prior', 0.473142489546478, array([0.06511628, 0.63691994, 0.24497041, 0.        , 0.01369863]), array([0.06034483, 0.61954365, 0.26744186, 0.        , 0.01785714]), array([0.06263982, 0.62811164, 0.2557134 , 0.        , 0.01550388]))
               precision    recall  f1-score   support

     negative      0.065     0.060     0.063       232
      neutral      0.637     0.620     0.628      2016
     positive      0.245     0.267     0.256       774
very negative      0.000     0.000     0.000        31
very positive      0.014     0.018     0.016        56

     accuracy                          0.473      3109
    macro avg      0.192     0.193     0.192      3109
 weighted avg      0.479     0.473     0.476      3109


Confusion matrix:
 [[  14  139   58    3    1]
 [ 154 1249  492   24   42]
 [  59  564  207    3   12]
 [   0   12    3    0    0]
 [   5   52   14    1    1]]


In [10]:
##Solution (c)
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='saga', max_iter = 1000)
lr_model = lr.fit(train_features, train_labels)
evaluation_summary("LR onehot", lr_model.predict(validation_features), validation_labels)

Evaluation for: LR onehot
Classifier has Acc= ('LR onehot', 0.752010292698617, array([0.26046512, 0.8827129 , 0.61893491, 0.2       , 0.34246575]), array([0.51851852, 0.78148984, 0.70201342, 0.42857143, 0.73529412]), array([0.34674923, 0.82902299, 0.65786164, 0.27272727, 0.46728972]))
               precision    recall  f1-score   support

     negative      0.260     0.519     0.347       108
      neutral      0.883     0.781     0.829      2215
     positive      0.619     0.702     0.658       745
very negative      0.200     0.429     0.273         7
very positive      0.342     0.735     0.467        34

     accuracy                          0.752      3109
    macro avg      0.461     0.633     0.515      3109
 weighted avg      0.790     0.752     0.766      3109


Confusion matrix:
 [[  56  151    7    1    0]
 [  46 1731  176    3    5]
 [   5  313  523    0    4]
 [   1   11    0    3    0]
 [   0    9   39    0   25]]


In [11]:
#solution (d)
from sklearn.feature_extraction.text import TfidfVectorizer
TFIDF_Vectoriser=TfidfVectorizer( binary=True, max_features = 5000 ) 

In [12]:
train_features_TFIDF = TFIDF_Vectoriser.fit_transform(train_data['body'])

# This creates input features for our classification on all subsets of our collection.
validation_features_TFIDF = TFIDF_Vectoriser.transform(validation_data['body'])
test_features_TFIDF = TFIDF_Vectoriser.transform(test_data['body'])

In [13]:
lr = LogisticRegression(solver='saga')
combined_model = lr.fit(train_features_TFIDF,train_labels)
evaluation_summary("LR TFIDF", lr.predict(validation_features_TFIDF), validation_labels) 

Evaluation for: LR TFIDF
Classifier has Acc= ('LR TFIDF', 0.7378578321003538, array([0.11162791, 0.93013768, 0.51597633, 0.        , 0.1369863 ]), array([0.68571429, 0.74297352, 0.71947195, 0.        , 0.76923077]), array([0.192     , 0.82608696, 0.60096485, 0.        , 0.23255814]))
               precision    recall  f1-score   support

     negative      0.112     0.686     0.192        35
      neutral      0.930     0.743     0.826      2455
     positive      0.516     0.719     0.601       606
very negative      0.000     0.000     0.000         0
very positive      0.137     0.769     0.233        13

     accuracy                          0.738      3109
    macro avg      0.339     0.583     0.370      3109
 weighted avg      0.837     0.738     0.773      3109


Confusion matrix:
 [[  24  185    6    0    0]
 [   8 1824  127    0    2]
 [   1  407  436    0    1]
 [   2   13    0    0    0]
 [   0   26   37    0   10]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
#solution (e)
from sklearn.svm import SVC
sv= SVC()
svc_model = sv.fit(train_features, train_labels)
evaluation_summary("LR onehot", svc_model.predict(validation_features), validation_labels)

Evaluation for: LR onehot
Classifier has Acc= ('LR onehot', 0.7224187841749758, array([0.01860465, 0.94390617, 0.46272189, 0.        , 0.        ]), array([0.66666667, 0.72107519, 0.72947761, 0.        , 0.        ]), array([0.0361991 , 0.81757951, 0.56625634, 0.        , 0.        ]))
               precision    recall  f1-score   support

     negative      0.019     0.667     0.036         6
      neutral      0.944     0.721     0.818      2567
     positive      0.463     0.729     0.566       536
very negative      0.000     0.000     0.000         0
very positive      0.000     0.000     0.000         0

     accuracy                          0.722      3109
    macro avg      0.285     0.423     0.284      3109
 weighted avg      0.859     0.722     0.773      3109


Confusion matrix:
 [[   4  208    3    0    0]
 [   1 1851  109    0    0]
 [   1  453  391    0    0]
 [   0   15    0    0    0]
 [   0   40   33    0    0]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
#Solution (f)

from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
nb_model = classifier.fit(train_features, train_labels)

validation_predicted_labels = nb_model.predict(validation_features)
evaluation_summary("One-hot NB",  validation_predicted_labels, validation_labels)

Evaluation for: One-hot NB
Classifier has Acc= ('One-hot NB', 0.6773882277259569, array([0.00930233, 0.92197858, 0.35029586, 0.        , 0.        ]), array([0.16666667, 0.6980695 , 0.58382643, 0.        , 0.        ]), array([0.01762115, 0.79455065, 0.43786982, 0.        , 0.        ]))
               precision    recall  f1-score   support

     negative      0.009     0.167     0.018        12
      neutral      0.922     0.698     0.795      2590
     positive      0.350     0.584     0.438       507
very negative      0.000     0.000     0.000         0
very positive      0.000     0.000     0.000         0

     accuracy                          0.677      3109
    macro avg      0.256     0.290     0.250      3109
 weighted avg      0.825     0.677     0.733      3109


Confusion matrix:
 [[   2  202   11    0    0]
 [   9 1808  144    0    0]
 [   0  549  296    0    0]
 [   1   12    2    0    0]
 [   0   19   54    0    0]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Q2
In this task you will improve the effectiveness of the LogisticRegression with TF-IDF vectorization from Q1.

i) **Parameter tuning** - Tune the parameters for both the vectorizer and classifier on the validation set (or using
CV-fold validation on the train).

● Classifier - Regularization C value (typical values might be powers of 10 (from 10^-3 to 10^5)

● Vectorizer - Parameters: sublinear_tf and max_features (vocabulary size) (in a range None to 50k)

● Select another parameter of your choice from the classifier or vectorizer


ii) **Error analysis** - Manually examine the predictions of your optimised classifier on the test set.

In [27]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import fbeta_score

def evaluation_summary1(description, predictions, true_labels):
  print("Evaluation for: " + description)
  precision = precision_score(predictions, true_labels,average='micro')
  recall = recall_score(predictions, true_labels,average=None)
  accuracy = accuracy_score(predictions, true_labels)
  f1 = fbeta_score(predictions, true_labels, 1,average=None) #1 means f_1 measure
  print("Classifier has Acc=", (description,accuracy,precision,recall,f1))
  print(classification_report(predictions, true_labels, digits=3, zero_division = 0))
  print('\nConfusion matrix:\n',confusion_matrix(true_labels, predictions)) # Note the order here is true, predicted


In [28]:
#solution (i)
from sklearn.feature_extraction.text import TfidfVectorizer
TFIDF_Vectoriser=TfidfVectorizer( binary=True,sublinear_tf = False, max_features = 1212 ) 

In [29]:
train_features_TFIDF = TFIDF_Vectoriser.fit_transform(train_data['body'])

# This creates input features for our classification on all subsets of our collection.
validation_features_TFIDF = TFIDF_Vectoriser.transform(validation_data['body'])
test_features_TFIDF = TFIDF_Vectoriser.transform(test_data['body'])

In [30]:
lr = LogisticRegression(solver='saga',C=10,penalty='l2')
combined_model = lr.fit(train_features_TFIDF,train_labels)
evaluation_summary1("LR TFIDF", lr.predict(validation_features_TFIDF), validation_labels) 

Evaluation for: LR TFIDF
Classifier has Acc= ('LR TFIDF', 0.7671276937922161, 0.7671276937922161, array([0.54330709, 0.78843568, 0.74717514, 0.5       , 0.67567568]), array([0.40350877, 0.83921756, 0.68126207, 0.28571429, 0.45454545]))
               precision    recall  f1-score   support

     negative      0.321     0.543     0.404       127
      neutral      0.897     0.788     0.839      2231
     positive      0.626     0.747     0.681       708
very negative      0.200     0.500     0.286         6
very positive      0.342     0.676     0.455        37

     accuracy                          0.767      3109
    macro avg      0.477     0.651     0.533      3109
 weighted avg      0.804     0.767     0.780      3109


Confusion matrix:
 [[  69  138    7    1    0]
 [  50 1759  144    2    6]
 [   5  305  529    0    6]
 [   3    9    0    3    0]
 [   0   20   28    0   25]]


In [32]:
#solution (ii)
lr = LogisticRegression(solver='saga',C=10,penalty='l1')
combined_model = lr.fit(train_features_TFIDF,train_labels)
evaluation_summary1("LR TFIDF", lr.predict(test_features_TFIDF), test_labels) 

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Evaluation for: LR TFIDF
Classifier has Acc= ('LR TFIDF', 0.7617031872509961, 0.7617031872509961, array([0.50438596, 0.80612626, 0.74025974, 0.33333333, 0.4375    ]), array([0.45098039, 0.83143903, 0.70470756, 0.32258065, 0.42168675]))
               precision    recall  f1-score   support

     negative      0.408     0.504     0.451       228
      neutral      0.858     0.806     0.831      2677
     positive      0.672     0.740     0.705      1001
very negative      0.312     0.333     0.323        30
very positive      0.407     0.438     0.422        80

     accuracy                          0.762      4016
    macro avg      0.532     0.564     0.546      4016
 weighted avg      0.773     0.762     0.766      4016


Confusion matrix:
 [[ 115  155    6    5    1]
 [ 103 2158  224   13   16]
 [   4  327  741    2   28]
 [   6   16    0   10    0]
 [   0   21   30    0   35]]


# Q3
In this task your goal is to add two features to (try to) improve sentiment polarity classification performance obtained
in Q2.3
You must implement and describe two new classifier features and add them to the tuned model from Q2.

In [33]:
#solution (3)
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import fbeta_score

def evaluation_summary1(description, predictions, true_labels):
  print("Evaluation for: " + description)
  precision = precision_score(predictions, true_labels,average=None)
  recall = recall_score(predictions, true_labels,average=None)
  accuracy = accuracy_score(predictions, true_labels)
  f1 = fbeta_score(predictions, true_labels, 1,average=None) #1 means f_1 measure
  print("Classifier has Acc=", (description,accuracy,precision,recall,f1))
  print(classification_report(predictions, true_labels, digits=3, zero_division = 0))
  print('\nConfusion matrix:\n',confusion_matrix(true_labels, predictions)) # Note the order here is true, predicted


In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
TFIDF_Vectoriser=TfidfVectorizer( binary=True,sublinear_tf = False , max_features = 1000 ) 

In [35]:
train_features_TFIDF = TFIDF_Vectoriser.fit_transform(train_data['in_reply_to'])

# This creates input features for our classification on all subsets of our collection.
validation_features_TFIDF = TFIDF_Vectoriser.transform(validation_data['in_reply_to'])
test_features_TFIDF = TFIDF_Vectoriser.transform(test_data['in_reply_to'])

In [36]:
lr = LogisticRegression(solver='saga',C=10,penalty='l2')
combined_model = lr.fit(train_features_TFIDF,train_labels)
evaluation_summary1("LR TFIDF", lr.predict(validation_features_TFIDF), validation_labels) 

Evaluation for: LR TFIDF
Classifier has Acc= ('LR TFIDF', 0.6307494371180444, array([0., 1., 0., 0., 0.]), array([0.        , 0.63074944, 0.        , 0.        , 0.        ]), array([0.        , 0.77357002, 0.        , 0.        , 0.        ]))
               precision    recall  f1-score   support

     negative      0.000     0.000     0.000         0
      neutral      1.000     0.631     0.774      3109
     positive      0.000     0.000     0.000         0
very negative      0.000     0.000     0.000         0
very positive      0.000     0.000     0.000         0

     accuracy                          0.631      3109
    macro avg      0.200     0.126     0.155      3109
 weighted avg      1.000     0.631     0.774      3109


Confusion matrix:
 [[   0  215    0    0    0]
 [   0 1961    0    0    0]
 [   0  845    0    0    0]
 [   0   15    0    0    0]
 [   0   73    0    0    0]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
train_features_TFIDF = TFIDF_Vectoriser.fit_transform(train_data['in_reply_to'])

# This creates input features for our classification on all subsets of our collection.
validation_features_TFIDF = TFIDF_Vectoriser.transform(validation_data['in_reply_to'])
test_features_TFIDF = TFIDF_Vectoriser.transform(test_data['in_reply_to'])

In [38]:
from sklearn.ensemble import RandomForestClassifier
RFC = RandomForestClassifier(n_estimators=100,min_samples_leaf=2)
combined_model = RFC.fit(train_features_TFIDF,train_labels)
evaluation_summary1("LR TFIDF", RFC.predict(test_features_TFIDF), test_labels) 

Evaluation for: LR TFIDF
Classifier has Acc= ('LR TFIDF', 0.625996015936255, array([0., 1., 0., 0., 0.]), array([0.        , 0.62599602, 0.        , 0.        , 0.        ]), array([0.        , 0.76998469, 0.        , 0.        , 0.        ]))
               precision    recall  f1-score   support

     negative      0.000     0.000     0.000         0
      neutral      1.000     0.626     0.770      4016
     positive      0.000     0.000     0.000         0
very negative      0.000     0.000     0.000         0
very positive      0.000     0.000     0.000         0

     accuracy                          0.626      4016
    macro avg      0.200     0.125     0.154      4016
 weighted avg      1.000     0.626     0.770      4016


Confusion matrix:
 [[   0  282    0    0    0]
 [   0 2514    0    0    0]
 [   0 1102    0    0    0]
 [   0   32    0    0    0]
 [   0   86    0    0    0]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Observations

The data has a neutral overall tone. Around 3000 posts are filled with good feelings. There are less than 1000 posts with "very positive" sentiments. The most recent post has received "very negative" feedback.


The data is split into three parts. Training, testing and validating. The model is based on the training data and the performance of the data is evaluated by predicting the test data. The different technology used is the virtual classifier, and logistic regression has the highest precision compared to other technologies. Use hot vector logistic regression, use TF IDF vector logistic regression, svc classifier and naive Bayes. The highest precision of the model classifier is found in logistic regression. One of the heat vectors has a precision of 75%, that is, the total percentage of the total correlation is 46%, and the total percentage after the classifier correctly obtains the total correlation is 63%. TF IDF has an overall accuracy of 73% and an accuracy of 33, and the F1 recovery rates are 58% and 37%. The virtual and out-of-tune classifiers and the most common strategies have an overall accuracy of about 63%. The hierarchical strategy virtual classifier has the lowest accuracy rate of 48%, and the accuracy rate, retrieval rate, and F1 rate are 19%, 19% and 19%.





The classifier of choice for the model and vectorization is Naive Bayes, because this algorithm is the best technique to use in classifying tests with multiple classes of variables. Compared to the performance of the different previous models, NB I will go to them. Its performance is better than the virtual classifier, but the precision is lower than the logic model and the CVC classifier, because the overall precision of the model is 67.7%, but the precision and recovery are better than the model with reduced precision and recovery by 25% and 29%, respectively, with an F1 score of 25%.